In [1]:
import deribit_data as dm
from option import Option
import numpy as np
from utils import *
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go

In [2]:
data = dm.Scraper(currency='BTC')

In [3]:
raw_df = data.collect_data(max_workers = 15, save_csv = False)

Data Collected


In [4]:
#raw_df.head()

In [5]:
option = Option(raw_df, 'C')

/Users/pierrebelamri/Documents/GitHub/deribit_scraping/utils.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mid_iv'] = (df.bid_iv + df.ask_iv)/2
/Users/pierrebelamri/Documents/GitHub/deribit_scraping/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_T'] = df['_T'].apply(to_ts) - datetime.datetime.timestamp(datetime.datetime.today())#btc_subset['timestamp'].apply(ms_to_s)
/Users/pierrebelamri/Documents/GitHub/deribit_scraping/utils.py:129: SettingWithCopyWarning: 
A value is trying 

In [6]:
option.init_merton(m = 0.4, lam =1, v = 0.3,  )

In [7]:
option.append_imp_vol_to_df()
option.append_loc_vol_to_df()

/Users/pierrebelamri/Documents/GitHub/deribit_scraping/option.py:164: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(num/denom)
/Users/pierrebelamri/Documents/GitHub/deribit_scraping/option.py:164: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(num/denom)


In [8]:
option.append_BS_price()

In [15]:
option.df.dropna(inplace=True)
#option.df = option.df.drop(option.df[option.df.IV_moneyness >= .5*10**6].index)

In [ ]:
start = time.process_time()
option.optimize_merton(tol=1e-20,max_iter = 1000)
t2 = time.process_time() - start
#print("max iter = 50, exec time = {}".format(t2))




In [28]:
x = option.df['S']/option.df['K']

#x = option.df['K']
y = option.df['_T']
z = option.df['I_VOL_BS']
z2 = option.df['mark_iv']/100


fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z , mode='markers', name='Implied vol')])
                      #go.Scatter3d(x=x, y=y, z=z2 , mode='markers', name='Implied vol deribit')])
"""
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z , mode='markers', name='Black&Scholes model Price'),
                      go.Scatter3d(x=x, y=y, z=z2, mode='markers', name='Real Market Price')])
"""

fig.update_scenes(xaxis_title_text='S/K', yaxis_title_text='Exp', zaxis_title_text='IV') 
fig.show()

In [29]:
rms = mean_squared_error(option.df['MERTON_PRICE'], option.df['mark_price'], squared=False)
print('Root Mean Squared Error = {}%'.format(round(rms*100,3)))

Root Mean Squared Error = 3.266%


In [30]:
option.df.dropna(inplace=True)
#option.df = option.df.drop(option.df[option.df.MERTON_PRICE >= .2].index)

In [32]:
option.append_Merton_price()

x = option.df['K']
y = option.df['_T']
z = option.df['BS_PRICE']
z2 = option.df['mark_price']
z3 = option.df['MERTON_PRICE']


#fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z , mode='markers', name='BS price')])

fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z , mode='markers', name='Black&Scholes model Price'),
                      go.Scatter3d(x=x, y=y, z=z2, mode='markers', name='Real Market Price'),
                      go.Scatter3d(x=x, y=y, z=z3, mode='markers', name='Merton Model Price')])


fig.update_scenes(xaxis_title_text='Strike', yaxis_title_text='Exp', zaxis_title_text='Price') 
fig.show()

rms_merton = mean_squared_error(option.df['mark_price'], option.df['MERTON_PRICE'], squared=False)
rms_bs = mean_squared_error(option.df['mark_price'], option.df['BS_PRICE'], squared=False)

print('Root Mean Squared Error | MERTON = {}%'.format(round(rms_merton*100,3)))
print('Root Mean Squared Error | B&S    = {}%'.format(round(rms_bs*100,3)))

Root Mean Squared Error | MERTON = 3.266%
Root Mean Squared Error | B&S    = 1.075%
